In [1]:
# PROJECT 2: Algorithmic Trading Strategy (Momentum)
# Author: Nazim Biplob
# Description: A Trend-Following Algo using Moving Averages on High-Frequency Data.

# 1. Setup
!pip install -q yfinance plotly
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# 2. Data Ingestion
ticker = "AAPL"
data = yf.download(ticker, start="2024-01-01", end="2026-01-01")

# Fix for yfinance formatting
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [col[0] for col in data.columns]

# 3. The Strategy (Math)
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['Signal'] = np.where(data['Close'] > data['SMA_50'], 1.0, 0.0)
data['Position'] = data['Signal'].diff()

# 4. Interactive Dashboard (Plotly)
fig = go.Figure()

# Candlestick Chart
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'], high=data['High'],
                low=data['Low'], close=data['Close'],
                name='Market Price'))

# Trend Line
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'],
                         mode='lines', name='50-Day Trend',
                         line=dict(color='orange', width=2)))

# Buy Signals
buy_signals = data[data['Position'] == 1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=data.loc[buy_signals.index]['SMA_50'],
                         mode='markers', name='Buy Signal',
                         marker=dict(symbol='triangle-up', size=15, color='green')))

# Sell Signals
sell_signals = data[data['Position'] == -1]
fig.add_trace(go.Scatter(x=sell_signals.index, y=data.loc[sell_signals.index]['SMA_50'],
                         mode='markers', name='Sell Signal',
                         marker=dict(symbol='triangle-down', size=15, color='red')))

# Styling
fig.update_layout(
    title='Algorithmic Trading Dashboard: Apple (AAPL)',
    yaxis_title='Price (USD)',
    template='plotly_dark',
    height=600
)

# 5. Save the interactive chart as a file
fig.write_html("apple_chart.html")
fig.show()

/tmp/ipython-input-189427148.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2024-01-01", end="2026-01-01")
[*********************100%***********************]  1 of 1 completed
